In [1]:
import requests
import json
import time
import pprint
import pandas as pd
from urllib import parse
pp = pprint.PrettyPrinter(indent=4)
request_header = {
                    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.164 Safari/537.36",
                    "Accept-Language": "ko,en-US;q=0.9,en;q=0.8,es;q=0.7",
                    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
                    "Origin": "https://developer.riotgames.com",
                    "X-Riot-Token": 'RGAPI-73e288ef-e47e-4716-a487-13c0df8f9c8e'
                }

# for문 진행률 확인 라이브러리
from tqdm import tqdm

In [57]:
def check(url):
    r = requests.get(url, headers=request_header)
    if r.status_code == 200: # response가 정상이면 바로 맨 밑으로 이동하여 정상적으로 코드 실행
        pass
    
    elif r.status_code == 429:
        print('api cost full : infinite loop start')
        start_time = time.time()
        
        while True: # 429error가 끝날 때까지 무한 루프
            if r.status_code == 429:

                print('try 10 second wait time')
                time.sleep(10)

                r = requests.get(url, headers=request_header)
                print(r.status_code)

            elif r.status_code == 200: #다시 response 200이면 loop escape
                print('total wait time : ', time.time() - start_time)
                print('recovery api cost')
                break
nick = "By 불비"
url_nick = f'https://kr.api.riotgames.com/lol/summoner/v4/summoners/by-name/{parse.quote(nick)}'
check(url_nick)
nick_to_id = requests.get(url_nick, headers=request_header).json()
input_id = nick_to_id['id']

url_game = f'https://kr.api.riotgames.com/lol/spectator/v4/active-games/by-summoner/{parse.quote(input_id)}'
check(url_game)
data = requests.get(url_game, headers=request_header).json()

dic = []

for i in range(0,10):
    champ_id = data['participants'][i]['championId']
    summoner_Id = data['participants'][i]['summonerId']
    champ_status_url = f"https://kr.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/{parse.quote(summoner_Id)}/by-champion/{champ_id}"
    check(champ_status_url)
    champ_status = requests.get(champ_status_url, headers=request_header).json()
    champ_point = champ_status['championPoints']
    dic.append(champ_id)
    dic.append(champ_point)


KeyError: 'championPoints'

In [55]:
from flask import Flask, render_template, request
import pickle
import sqlite3
import pandas as pd
from xgboost import XGBClassifier
#def create_app():

app = Flask(__name__)

model = None

with open('model.pkl', 'rb') as file:
    model = pickle.load(file)
columns = ['1', '1score', '2', '2score', '3', '3score', '4', '4score', '5', '5score', '6', '6score', '7', '7score', '8', '8score', '9', '9score', '10', '10score']
df = pd.DataFrame(columns = columns)
input_data = {
              '1' : dic[0],
              '1score' : dic[1],
              '2' : dic[2],
              '2score' :dic[3],
              '3' : dic[4],
              '3score' : dic[5],
              '4' : dic[6],
              '4score' : dic[7],
              '5' : dic[8],
              '5score' : dic[9],
              '6' : dic[10],
              '6score' : dic[11],
              '7' : dic[12],
              '7score' : dic[13],
              '8' : dic[14],
              '8score' : dic[15],
              '9' : dic[16],
              '9score' : dic[17],
              '10' : dic[18],
              '10score' : dic[19],
              }
X_test = df.append(input_data, ignore_index=True)
X_test = X_test.astype(int)
y_pred = model.predict(X_test)
c=model.predict_proba(X_test).reshape(-1,1)

In [56]:
c


array([[0.32953662],
       [0.6704634 ]], dtype=float32)

In [ ]:
gameId = list(gameId)
for i in range(len(gameId)):
    input_data = {'gameId' : gameId[i]}
    gameId_master.append(input_data, ignore_index = True)

In [ ]:
gameId_master.to_csv("gameId_challanger.csv", header=True, index = False)

In [ ]:
match_grandmaster = pd.DataFrame(columns = ['teamId','win','firstBlood','firstTower','firstInhibitor','firstBaron','firstDragon','firstRiftHerald','championKills', 'towerKills','inhibitorKills','baronKills','dragonKills','riftHeraldKills','bans','gameId'])

In [ ]:
num = 0

for a in tqdm(range(len(gameId))):
    for b in range(len(gameId[a])):
                
        url4 =  'https://asia.api.riotgames.com/lol/match/v5/matches/' + str(gameId[a][b])
        r = requests.get(url4,headers = request_header)
        
                
        if r.status_code == 200: # response가 정상이면 바로 맨 밑으로 이동하여 정상적으로 코드 실행
            pass

        elif r.status_code == 429:
            print('api cost full : infinite loop start')
            print('loop location : ',i)
            start_time = time.time()
    
            while True: # 429error가 끝날 때까지 무한 루프
                if r.status_code == 429:

                    print('try 10 second wait time')
                    time.sleep(10)

                    r = requests.get(url4, headers = request_header)
                    print(r.status_code)

                elif r.status_code == 200: #다시 response 200이면 loop escape
                    print('total wait time : ', time.time() - start_time)
                    print('recovery api cost')
                    break
                    
        r = requests.get(url4, headers = request_header).json()          
        
        try:
            if r['info']['gameMode'] == 'CLASSIC':
                for i in [0,1]:
                    if r['info']['teams'][i]['win'] == False:
                            input_data = {
                                'teamId': str(r['info']['teams'][i]['teamId']),
                                'win':r['info']['teams'][i]['win'],
                                'firstBlood':r['info']['teams'][i]['objectives']['champion']['first'],
                                'firstTower':r['info']['teams'][i]['objectives']['tower']['first'],
                                'firstInhibitor':r['info']['teams'][i]['objectives']['inhibitor']['first'],
                                'firstBaron':r['info']['teams'][i]['objectives']['baron']['first'],
                                'firstDragon':r['info']['teams'][i]['objectives']['dragon']['first'],
                                'firstRiftHerald':r['info']['teams'][i]['objectives']['riftHerald']['first'],
                                'championKills' : r['info']['teams'][i]['objectives']['champion']['kills'],
                                'towerKills':r['info']['teams'][i]['objectives']['tower']['kills'],
                                'inhibitorKills':r['info']['teams'][i]['objectives']['inhibitor']['kills'],
                                'baronKills':r['info']['teams'][i]['objectives']['baron']['kills'],
                                'dragonKills':r['info']['teams'][i]['objectives']['dragon']['kills'],
                                'riftHeraldKills':r['info']['teams'][i]['objectives']['riftHerald']['kills'],
                                'bans' :r['info']['teams'][i]['bans'],
                                'gameId': gameId[a][b]
                            }

                            match_grandmaster = match_grandmaster.append(input_data, ignore_index = True)
                            
                    elif r['info']['teams'][i]['win'] == True:
                            input_data = {
                                'teamId': str(r['info']['teams'][i]['teamId']),
                                'win':r['info']['teams'][i]['win'],
                                'firstBlood':r['info']['teams'][i]['objectives']['champion']['first'],
                                'firstTower':r['info']['teams'][i]['objectives']['tower']['first'],
                                'firstInhibitor':r['info']['teams'][i]['objectives']['inhibitor']['first'],
                                'firstBaron':r['info']['teams'][i]['objectives']['baron']['first'],
                                'firstDragon':r['info']['teams'][i]['objectives']['dragon']['first'],
                                'firstRiftHerald':r['info']['teams'][i]['objectives']['riftHerald']['first'],
                                'championKills' : r['info']['teams'][i]['objectives']['champion']['kills'],
                                'towerKills':r['info']['teams'][i]['objectives']['tower']['kills'],
                                'inhibitorKills':r['info']['teams'][i]['objectives']['inhibitor']['kills'],
                                'baronKills':r['info']['teams'][i]['objectives']['baron']['kills'],
                                'dragonKills':r['info']['teams'][i]['objectives']['dragon']['kills'],
                                'riftHeraldKills':r['info']['teams'][i]['objectives']['riftHerald']['kills'],
                                'bans' :r['info']['teams'][i]['bans'],
                                'gameId': gameId[a][b]
                            }
                                

                            match_grandmaster = match_grandmaster.append(input_data, ignore_index = True)
    
        except:
            print("403 에러.api 만료")

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:03<00:00,  1.89it/s]


In [ ]:
match_grandmaster

,teamId,win,firstBlood,firstTower,firstInhibitor,firstBaron,firstDragon,firstRiftHerald,championKills,towerKills,inhibitorKills,baronKills,dragonKills,riftHeraldKills,bans,gameId
0,100,True,False,True,True,False,False,True,40,8,2,1,1,1,"[{'championId': 711, 'pickTurn': 1}, {'champio...",KR_5685811134
1,200,False,True,False,False,True,True,False,42,9,4,2,5,1,"[{'championId': -1, 'pickTurn': 6}, {'champion...",KR_5685811134
2,100,True,True,True,True,True,True,False,28,10,2,1,1,1,"[{'championId': 64, 'pickTurn': 1}, {'champion...",KR_5684039329
3,200,False,False,False,False,False,False,True,16,2,0,0,3,1,"[{'championId': 154, 'pickTurn': 6}, {'champio...",KR_5684039329
4,100,True,True,True,True,False,True,True,29,8,1,0,1,2,"[{'championId': 89, 'pickTurn': 1}, {'champion...",KR_5686407086
5,200,False,False,False,False,False,False,False,19,2,0,0,0,0,"[{'championId': 58, 'pickTurn': 6}, {'champion...",KR_5686407086
6,100,False,False,True,False,False,True,True,36,4,0,0,2,2,"[{'championId': 76, 'pickTurn': 1}, {'champion...",KR_5685150228
7,200,True,True,False,True,True,False,False,34,10,2,1,2,0,"[{'championId': 51, 'pickTurn': 6}, {'champion...",KR_5685150228
8,100,True,False,True,True,False,True,True,35,6,1,0,2,2,"[{'championId': -1, 'pickTurn': 1}, {'champion...",KR_5685771909
9,200,False,True,False,False,False,False,False,15,0,0,0,0,0,"[{'championId': 104, 'pickTurn': 6}, {'champio...",KR_5685771909


In [ ]:
match_grandmaster.to_csv("match_challanger.csv", header=True, index = False)